# 步骤三：创建数据库

In [1]:
import sqlite3

# 连接到 xianyu 数据库
conn = sqlite3.connect('xianyu.db')
cursor = conn.cursor()

In [9]:
# 打印指定表的结构（字段信息）
def print_table_schema(table_name):
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()

    # 获取列的最大宽度
    max_col_name_len = max(len(col[1]) for col in columns)
    max_type_len = max(len(col[2]) for col in columns)

    # 打印表头
    print(f"\nTable {table_name}\'s schema: ")
    print(f"{'Name'.ljust(max_col_name_len)} | {'Type'.ljust(max_type_len)} | Primary Key")
    print("-" * (max_col_name_len + 3 + max_type_len + 3 + 4))

    # 打印每一列的信息
    for column in columns:
        col_name = column[1]
        col_type = column[2]
        is_primary_key = "Yes" if column[5] else "No"
        print(f"{col_name.ljust(max_col_name_len)} | {col_type.ljust(max_type_len)} | {is_primary_key}")

In [ ]:
## 创建用户(User)表

In [10]:
# 创建用户表 (User)
cursor.execute('''
CREATE TABLE IF NOT EXISTS User (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL,
    password_hash TEXT NOT NULL,
    phone TEXT UNIQUE,
    email TEXT UNIQUE,
    credit_score INTEGER DEFAULT 0,
    registration_date TEXT DEFAULT (datetime('now')),
    last_login TEXT,
    status TEXT DEFAULT 'active'
)
''')
print_table_schema('User')


Table User's schema: 
Name              | Type    | Primary Key
----------------------------------
user_id           | INTEGER | Yes
username          | TEXT    | No
password_hash     | TEXT    | No
phone             | TEXT    | No
email             | TEXT    | No
credit_score      | INTEGER | No
registration_date | TEXT    | No
last_login        | TEXT    | No
status            | TEXT    | No


## 创建商品(Product)表
说明：`FOREIGN KEY (seller_id) REFERENCES User(user_id) ON DELETE CASCADE`O定义了一个外键约束，并且指定了当被引用的记录（即 User 表中的记录）被删除时所有在子表（如 Product 表）中通过外键与该记录相关联的行也会自动被删除。这是一种级联删除操作。

In [11]:
# 创建商品表 (Product)
cursor.execute('''
CREATE TABLE IF NOT EXISTS Product (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    seller_id INTEGER NOT NULL,
    title TEXT NOT NULL,
    description TEXT,
    price REAL NOT NULL,
    original_price REAL,
    condition TEXT CHECK(condition IN ('new', 'like new', 'good', 'fair', 'poor')),
    location TEXT,
    post_date TEXT DEFAULT (datetime('now')),
    status TEXT DEFAULT 'available' CHECK(status IN ('available', 'reserved', 'sold', 'removed')),
    view_count INTEGER DEFAULT 0,
    fav_count INTEGER DEFAULT 0,
    FOREIGN KEY (seller_id) REFERENCES User(user_id) ON DELETE CASCADE
)
''')
print_table_schema('Product')


Table Product's schema: 
Name           | Type    | Primary Key
-------------------------------
product_id     | INTEGER | Yes
seller_id      | INTEGER | No
title          | TEXT    | No
description    | TEXT    | No
price          | REAL    | No
original_price | REAL    | No
condition      | TEXT    | No
location       | TEXT    | No
post_date      | TEXT    | No
status         | TEXT    | No
view_count     | INTEGER | No
fav_count      | INTEGER | No


## 创建订单(Order)表
说明：注意 Order 是一个保留关键字，这里必须通过加下划线等方式避免。

In [14]:
# 创建订单表 (Order)
cursor.execute('''
CREATE TABLE IF NOT EXISTS Order_ (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_id INTEGER NOT NULL,
    buyer_id INTEGER NOT NULL,
    seller_id INTEGER NOT NULL,
    order_date TEXT DEFAULT (datetime('now')),
    price REAL NOT NULL,
    status TEXT DEFAULT 'pending' CHECK(status IN ('pending', 'paid', 'shipped', 'completed', 'cancelled')),
    payment_method TEXT,
    shipping_address TEXT,
    tracking_number TEXT,
    FOREIGN KEY (product_id) REFERENCES Product(product_id) ON DELETE CASCADE,
    FOREIGN KEY (buyer_id) REFERENCES User(user_id) ON DELETE CASCADE,
    FOREIGN KEY (seller_id) REFERENCES User(user_id) ON DELETE CASCADE
)
''')
print_table_schema('Order_')


Table Order_'s schema: 
Name             | Type    | Primary Key
---------------------------------
order_id         | INTEGER | Yes
product_id       | INTEGER | No
buyer_id         | INTEGER | No
seller_id        | INTEGER | No
order_date       | TEXT    | No
price            | REAL    | No
status           | TEXT    | No
payment_method   | TEXT    | No
shipping_address | TEXT    | No
tracking_number  | TEXT    | No


## 创建消息(Message)表

In [15]:
# 创建消息表 (Message)
cursor.execute('''
CREATE TABLE IF NOT EXISTS Message (
    message_id INTEGER PRIMARY KEY AUTOINCREMENT,
    sender_id INTEGER NOT NULL,
    receiver_id INTEGER NOT NULL,
    product_id INTEGER,
    content TEXT NOT NULL,
    send_time TEXT DEFAULT (datetime('now')),
    is_read INTEGER DEFAULT 0,
    FOREIGN KEY (sender_id) REFERENCES User(user_id) ON DELETE CASCADE,
    FOREIGN KEY (receiver_id) REFERENCES User(user_id) ON DELETE CASCADE,
    FOREIGN KEY (product_id) REFERENCES Product(product_id) ON DELETE SET NULL
)
''')
print_table_schema('Message')


Table Message's schema: 
Name        | Type    | Primary Key
----------------------------
message_id  | INTEGER | Yes
sender_id   | INTEGER | No
receiver_id | INTEGER | No
product_id  | INTEGER | No
content     | TEXT    | No
send_time   | TEXT    | No
is_read     | INTEGER | No


## 创建收藏(Favorite)表
说明：`UNIQUE (user_id, product_id)` 约束确保了同一个用户不能收藏同一件商品多次。

In [16]:
# 创建收藏表 (Favorite)
cursor.execute('''
CREATE TABLE IF NOT EXISTS Favorite (
    favorite_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    add_date TEXT DEFAULT (datetime('now')),
    UNIQUE(user_id, product_id),
    FOREIGN KEY (user_id) REFERENCES User(user_id) ON DELETE CASCADE,
    FOREIGN KEY (product_id) REFERENCES Product(product_id) ON DELETE CASCADE
)
''')
print_table_schema('Favorite')


Table Favorite's schema: 
Name        | Type    | Primary Key
----------------------------
favorite_id | INTEGER | Yes
user_id     | INTEGER | No
product_id  | INTEGER | No
add_date    | TEXT    | No
